### Этап 1. Получение ссылок на книги автора

Для поиска всех книг автора мы будем пользоваться поисковой строкой сайта. Обратите внимание на формат URL-ов запросов:
```
https://www.chitai-gorod.ru/search/result.php?q={author}&type=author
```
где вместо `{author}` подставляется имя автора.

Рекомендуемые варианты значения для `{author}`: `Фрай М.`, `Хантер Э.`, `Емец Д.`.

**Замечание:** URL [не позволяет](https://ru.wikipedia.org/wiki/URL#%D0%9A%D0%BE%D0%B4%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_URL) использовать кириллические символы, поэтому для кодирования части URL-а, содержащей кириллицу, стоит воспользоваться функцией `urllib.parse.quote` с использованием, в данном случае, кодировки `windows-1251`.

Попробуйте пролистать [страничку](https://www.chitai-gorod.ru/search/result.php?q=%D4%F0%E0%E9%20%CC.&type=author) вниз, и вы увидите, что новые книги подгружаются динамически. В некоторый момент появляется кнопка "Показать ещё", нажав на которую можно дополнительно подгрузить выдачу.

Чтобы получить все ссылки на карточки с книгами, нужно подгрузить полную выдачу по запросу.

Существует несколько способов получить ссылку на карточку:
1. получить атрибут `data-product` одного из тегов с классом `product-card` и использовать знание о формате URL-ов страничек с книгами;
2. непосредственно найти ссылку.

In [2]:
import re
import requests
from tqdm import tqdm
from multiprocessing.dummy import Pool as ThreadPool
import urllib
import pandas as pd
from bs4 import BeautifulSoup

## Don't work

In [3]:
%%time
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' +
                         'AppleWebKit/537.36 (KHTML, like Gecko) ' +
                         'Chrome/42.0.2311.90 Safari/537.36'}

names_authors = ["Пушкин А."]
names_url = ['https://www.chitai-gorod.ru/search/result.php?q={}&type=author'.format(urllib.parse.quote(name, encoding="CP1251")) for name in names_authors]
t = ThreadPool(16)
r = t.map(lambda name_url: requests.get(name_url, headers=headers), names_url)
t.close()
t.join()

CPU times: user 29.1 ms, sys: 17.1 ms, total: 46.2 ms
Wall time: 408 ms


In [4]:
one = r[0].content
soup = BeautifulSoup(one, "html.parser")
# print(soup.prettify()

## Selenium
http://selenium-python.readthedocs.io/

https://kreisfahrer.gitbooks.io/selenium-webdriver/

In [5]:
from selenium import webdriver
import html
from time import sleep

In [6]:
with webdriver.Firefox() as driver:
    driver.get("https://mail.ru/")
    sleep(3)

In [7]:
url_current = names_url[0]
url_current

'https://www.chitai-gorod.ru/search/result.php?q=%CF%F3%F8%EA%E8%ED%20%C0.&type=author'

In [8]:
import random

def random_sleep(offset=1.5, length=4):
    sleep(random.random() * length + offset)
    
def hasclass(driver, path):
    try:
        driver.find_element_by_class_name(path)
        return True
    except:
        return False
    
def get_books_info(driver, urls, t_sleep=1):
    result = []
    for url in urls:
        driver.get(url)
        random_sleep()
        SCROLL_PAUSE_TIME = 3

        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                # [ Show more ]
                try:
                    more = driver.find_element_by_class_name('js__show-more-cards')
                    print("Press 'Show_more' = ", hasclass(driver,'js__show-more-cards'))
                    more.click()
                except:
                    break
                random_sleep(1, 1)
            last_height = new_height

        items = driver.find_elements_by_class_name("product-card")
        for item in items:
            book_url_info = {
                'id': item.get_attribute("data-product")}
        
            if book_url_info['id'] is None:
                continue
            book_url_info['id'] = html.unescape(book_url_info['id'])
            result.append(book_url_info)
        
    return result


with webdriver.Firefox() as driver:
    result = get_books_info(driver, names_url)

Press 'Show_more' =  True
Press 'Show_more' =  True
Press 'Show_more' =  True
Press 'Show_more' =  True
Press 'Show_more' =  True
Press 'Show_more' =  True
Press 'Show_more' =  True
Press 'Show_more' =  True
Press 'Show_more' =  True
Press 'Show_more' =  True
Press 'Show_more' =  True


In [9]:
result = pd.DataFrame(result)
print(result.shape)
result.head()

(629, 1)


,id
0,1086411
1,775884
2,775883
3,766500
4,766489


### Этап 2. Получение информации о книгах

Рассмотрим в качестве примера [карточку](https://www.chitai-gorod.ru/catalog/book/1059170/) книги Макса Фрая "Мертвый ноль".
![скриншот](images/1059170.png)


На скриншоте выделены 4 области, из каждой области требуется извлечь следующие элементы:
1. поле "ID карточки";
2. поля "Название", "Автор", "Рейтинг", "Голоса";
3. поле "Цена";
4. вся таблица.

Дополнительно требуется создать поле "Обложка", которое будет хранить ссылку на обложку книги (расположена слева на скриншоте).

Таким образом карточка представима в виде следующего словаря:
```json
{
    "ID карточки": 1059170,
    "Название": "Мертвый ноль",
    "Автор": "Фрай М.",
    "Рейтинг": 4.0,
    "Голоса": 16,
    "Цена": 375,
    "Серия": "Сновидения Ехо",
    "Издательство": "АСТ",
    "Год издания": 2018,
    "Кол-во страниц": 448,
    "ISBN": "9785171089733",
    "Тираж": 30000,
    "Формат": "20.5 x 13 x 2.5",
    "Тип обложки": "Твердая бумажная",
    "Возраст": "16+",
    "ID товара": 2659485,
    "Обложка": "https://img-gorod.ru/upload/iblock/aec/aec2cfaece8a6190f319f1853cad7cf5.jpg"
}
```

Предположим, что у нас есть функция `extract_book_info(card_id)`, которая для карточки с номером `card_id` возвращает описанный выше словарь. Тогда требуемую таблицу можно получить следующим образом:
```python
import pandas as pd

result = list(map(extract_book_info, card_ids))

df = pd.DataFrame(result)
df.sort_values(by='ID карточки', inplace=True)

with open('data/hw_4.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)
```

## Beautiful Soup
https://www.crummy.com/software/BeautifulSoup/bs4/doc/

![Beautiful Soup](images/bsoup.jpg)

In [10]:
from bs4 import BeautifulSoup

In [11]:
list_of_url = ["https://www.chitai-gorod.ru/catalog/book/{}/?watch_fromlist=search_result".
               format(id) for id in list(result.id.values)]

In [12]:
%%time
url = "https://www.chitai-gorod.ru/catalog/book/{}/?watch_fromlist=search_result".format(result.id[0])
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' +
                         'AppleWebKit/537.36 (KHTML, like Gecko) ' +
                         'Chrome/42.0.2311.90 Safari/537.36'}

r = requests.get(url, headers=headers)

CPU times: user 23.2 ms, sys: 7.56 ms, total: 30.8 ms
Wall time: 611 ms


In [13]:
book_html = r.text
soup = BeautifulSoup(book_html, 'html.parser')

In [14]:
def re_sub(string):
    return re.sub("\s+", ' ', string.strip())
def re_sub_one(string):
    return re.sub("\s", ' ', string.strip()).split()[0]
def re_sub_two(string):
    string = re.sub("\s", ' ', string.strip()).split()[1]
    return re.sub("\D", '', string.strip())
def re_sub_n(string):
     return re.sub("\D", '', string.strip())


In [15]:
film_info = {
    "ID карточки": '',
    "Название": '',
    "Автор": '',
    "Рейтинг": '',
    "Голоса": '',
    "Цена": '',
    "Серия": '',
    "Издательство": '',
    "Год издания": '',
    "Кол-во страниц": '',
    "ISBN": '',
    "Тираж": '',
    "Формат": '',
    "Тип обложки": '',
    "Возраст": '',
    "ID товара": '',
    "Обложка": '',
    "Переводчик": '',
    "Редактор": '',
    "Художник": ''
}

In [16]:
film_info['ID карточки'] = result.id[0]
film_info['Название'] = re_sub(soup.find('h1', class_="product__title js-analytic-product-title").text)
film_info['Автор'] = re_sub(soup.find('a', class_= "link product__author").text)
film_info['Рейтинг'] = re_sub_one(soup.find('div',class_="rating").text)
film_info['Голоса'] = re_sub_two(soup.find('div',class_="rating").text)
film_info['Цена'] = re_sub_n(soup.find('div', class_='price').text)
film_info['Обложка'] = soup.find('div', class_='product__image').find("img").attrs['src']

film_info

{'ID карточки': '1086411',
 'Название': 'Борис Годунов. Моцарт и Сальери. Пир во время чумы',
 'Автор': 'Пушкин А.',
 'Рейтинг': '0',
 'Голоса': '0',
 'Цена': '114',
 'Серия': '',
 'Издательство': '',
 'Год издания': '',
 'Кол-во страниц': '',
 'ISBN': '',
 'Тираж': '',
 'Формат': '',
 'Тип обложки': '',
 'Возраст': '',
 'ID товара': '',
 'Обложка': 'https://img-gorod.ru/upload/iblock/927/9276b2b4396e0dd0209428a25b3adeda.jpg',
 'Переводчик': '',
 'Редактор': '',
 'Художник': ''}

In [17]:
table = soup.find('div', class_='product__props')
rows = table.find_all('div', class_="product-prop")

In [18]:
data = [list(map(lambda x: re.sub("\s+", ' ', x.text.strip()), row.find_all('div'))) for row in rows]
print(data)
for property_book in data:
    if property_book[0] == "ID товара:":
        film_info["ID товара"] = property_book[1]
    else:
        film_info[property_book[0]] = property_book[1]
film_info

[['Серия', 'Классика'], ['Издательство', 'Эксмо'], ['Год издания', '2019'], ['Кол-во страниц', '160'], ['ISBN', '9785040983254'], ['Тираж', '5000'], ['Формат', '20.7 x 13 x 1'], ['Тип обложки', 'Твердая глянцевая'], ['Возраст', '6+'], ['ID товара:', '2688100']]


{'ID карточки': '1086411',
 'Название': 'Борис Годунов. Моцарт и Сальери. Пир во время чумы',
 'Автор': 'Пушкин А.',
 'Рейтинг': '0',
 'Голоса': '0',
 'Цена': '114',
 'Серия': 'Классика',
 'Издательство': 'Эксмо',
 'Год издания': '2019',
 'Кол-во страниц': '160',
 'ISBN': '9785040983254',
 'Тираж': '5000',
 'Формат': '20.7 x 13 x 1',
 'Тип обложки': 'Твердая глянцевая',
 'Возраст': '6+',
 'ID товара': '2688100',
 'Обложка': 'https://img-gorod.ru/upload/iblock/927/9276b2b4396e0dd0209428a25b3adeda.jpg',
 'Переводчик': '',
 'Редактор': '',
 'Художник': ''}

In [22]:
%%time
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' +
                         'AppleWebKit/537.36 (KHTML, like Gecko) ' +
                         'Chrome/42.0.2311.90 Safari/537.36'}
t = ThreadPool(16)
r = t.map(lambda id: (id, requests.get("https://www.chitai-gorod.ru/catalog/book/{}/?watch_fromlist=search_result".format(id), headers=headers)), list(result.id.values))
t.close()
t.join()

CPU times: user 18.4 s, sys: 775 ms, total: 19.2 s
Wall time: 33.3 s


In [23]:
# create def extract_book_info
def extract_book_info(book):
    book_html = book[1].text
    soup = BeautifulSoup(book_html, 'html.parser')
    film_info = {
    "ID карточки": '',
    "Название": '',
    "Автор": '',
    "Рейтинг": '',
    "Голоса": '',
    "Цена": '',
    "Серия": '',
    "Издательство": '',
    "Год издания": '',
    "Кол-во страниц": '',
    "ISBN": '',
    "Тираж": '',
    "Формат": '',
    "Тип обложки": '',
    "Возраст": '',
    "ID товара": '',
    "Обложка": '',
    "Переводчик": '',
    "Редактор": '',
    "Художник": '',
    "Составитель": ''
    }
    film_info['ID карточки'] = book[0]
    film_info['Название'] = re_sub(soup.find('h1', class_="product__title js-analytic-product-title").text)
    film_info['Автор'] = re_sub(soup.find('a', class_= "link product__author").text)
    film_info['Рейтинг'] = re_sub_one(soup.find('div',class_="rating").text)
    film_info['Голоса'] = re_sub_two(soup.find('div',class_="rating").text)
    try:
        film_info['Цена'] = re_sub_n(soup.find('div', class_='price').text)
    except:
        film_info['Цена'] = ''
    table = soup.find('div', class_='product__props')
    rows = table.find_all('div', class_="product-prop")
    data = [list(map(lambda x: re.sub("\s+", ' ', x.text.strip()), row.find_all('div'))) for row in rows]
    for property_book in data:
        if property_book[0] == "ID товара:":
            film_info["ID товара"] = property_book[1]
        else:
            film_info[property_book[0]] = property_book[1]
    film_info['Обложка'] = soup.find('div', class_='product__image').find("img").attrs['src']
    return film_info

In [24]:
card = r
result = list(map(extract_book_info, card))
df = pd.DataFrame(result)
df.sort_values(by='ID карточки', inplace=True)

In [25]:
with open('hw_4_my.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)

In [26]:
df.head(22)

,ID карточки,ID товара,ISBN,Автор,Возраст,Год издания,Голоса,Издательство,Кол-во страниц,Название,...,Переводчик,Редактор,Рейтинг,Серия,Составитель,Тип обложки,Тираж,Формат,Художник,Цена
133,1004316,1347326,5170071795,Пушкин А.,,2002,0,АСТ,192,Пушкин Стихотворения,...,,,0,Хрестоматия школьника,,Твердая глянцевая,,20.7 x 13.3 x 1.5,,74
132,1005450,2610031,9785995132998,Пушкин А.,6+,2017,0,Стрекоза,80,Сказки,...,,,0,Внеклассное чтение,,Мягкая бумажная,,21 x 14 x 0.4,Бедарев Г.,68
131,1006688,2611100,9785699780938,Пушкин А.,0+,2017,0,Эксмо,43,Сказка о рыбаке и рыбке,...,,Талалаева Е.,0,Самые лучшие сказки,,Твердая глянцевая,5000,20.3 x 14.5 x 0.8,,141
130,1009569,2613592,9785353085843,Пушкин А.,0+,2017,0,Росмэн,32,Сказка о золотом петушке,...,,Конча Н.,0,Детская библиотека РОСМЭН,,Твердая глянцевая,,20 x 13.5 x 0.5,,72
129,1010177,2614206,9785179832010,Пушкин А.,12+,2017,0,АСТ,222,Дубровский. Повести Белкина,...,,,0,Школьное чтение,,Твердая бумажная,3000,20.5 x 13 x 1.3,,162
128,1010178,2614207,9785179832027,Пушкин А.,12+,2017,0,АСТ,222,Дубровский. Повести Белкина,...,,,0,Классика для школьников,,Твердая бумажная,2000,20.5 x 13 x 1.3,,178
127,1013331,2616142,9785389133136,Пушкин А.,0+,2018,0,Махаон,176,Повести Белкина. Дубровский,...,,Бирюкова А.Ю.,0,,,Твердая бумажная,5000,24.4 x 20 x 1.8,Иткин Анатолий,408
126,1014019,2616797,9785171057824,Пушкин А.,12+,2017,0,АСТ,352,Пиковая дама. Повести и поэмы,...,,,0,Эксклюзив: Русская классика,,Мягкая бумажная,5000,18 x 11.6 x 1.8,,138
125,1014441,2617203,9785040894284,Пушкин А.,16+,2017,0,Издательство Э,544,Стихотворения. Сказки. Поэмы,...,,,0,100 главных книг,,Твердая бумажная,2000,20.8 x 13.5 x 2.8,,304
124,1015173,2617918,9785040900091,Пушкин А.,16+,2017,0,Издательство Э,384,Евгений Онегин,...,,,0,Всемирная библиотека поэзии,,Твердая бумажная,3000,17 x 17.5 x 2,,203


In [ ]:
# Выкачать картинки
for i in range(df.shape[0]):
    img = df['Обложка'][i]
    p = requests.get(img)
    path = "book_img_my/{}.jpg".format(i)
    out = open(path, "wb")
    out.write(p.content)
    out.close()

Мой результат работы программы можно найти [здесь](hw_4.csv).